In [ ]:
#****************
#change all these except for BUCKET_LOC
#****************
PROJECT_ID = 'change me'
DATASET_ID = 'change me'
OJBECT_TABLE_NAME= 'obj_cloud_ai_images'
CONN_NAME= 'conn_cloud_ai'
CONN_SERVICE_ACCOUNT = ""
REGION = "US"
VISION_MODEL_NAME = 'bqml-cloud-ai-vision'
TRANSLATE_MODEL_NAME = 'bqml-cloud-ai-translate'
TEXT_MODEL_NAME = 'bqml-cloud-ai-text'



Demo of BQML cloud-ai API functionality.

Objective of Demo:  Using unstructured data in BQ Data Analytics workflows

Main steps:
1.   create object table
2.   Call vision ai (ML.ANNOTATE_IMAGE) to do OCR
3.   Call translate ai (ML.TRANSLATE) to detect language of text
4.   Call translate ai (ML.TRANSLATE) to translate non-english records to english
5.   Call NLP ai (ML.UNDERSTAND_TEXT) to classify text



In [ ]:
!pip install google-cloud-bigquery-connection


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 KB 1.4 MB/s eta 0:00:00


In [ ]:
from google.cloud import bigquery, bigquery_connection_v1

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
!gcloud config set project {PROJECT_ID}
shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = shell_output[0]
print("Project ID now set to:", PROJECT_ID)

Updated property [core/project].
Project ID now set to: bigquerytestdefault


In [ ]:
#you may need to enable apis
#!gcloud services enable bigqueryconnection.googleapis.com

In [ ]:
#Create BQ Connection
from google.cloud import bigquery as bq
from google.cloud import bigquery_connection_v1 as bq_connection
import googleapiclient.discovery

client = bq_connection.ConnectionServiceClient()
new_conn_parent="projects/{}/locations/{}".format(PROJECT_ID, "US")
exists_conn_parent="projects/{}/locations/{}/connections/{}".format(PROJECT_ID, "US", CONN_NAME)
cloud_resource_properties = bq_connection.CloudResourceProperties({})

try:
  request = client.get_connection(request=bq_connection.GetConnectionRequest(name=exists_conn_parent))
  CONN_SERVICE_ACCOUNT = "serviceAccount:{}".format(request.cloud_resource.service_account_id)
except Exception as e:
  connection = bq_connection.types.Connection(
          {
              "friendly_name": CONN_NAME,
              "cloud_resource": cloud_resource_properties
          }
      )
  request = bq_connection.CreateConnectionRequest(
          {
              "parent": new_conn_parent,
              "connection_id": CONN_NAME,
              "connection": connection
          }
      )
  response = client.create_connection(request)
  CONN_SERVICE_ACCOUNT = "serviceAccount:{}".format(response.cloud_resource.service_account_id)
print(CONN_SERVICE_ACCOUNT)

serviceAccount:bqcx-442188550395-8b9b@gcp-sa-bigquery-condel.iam.gserviceaccount.com


In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/serviceusage.serviceUsageConsumer'

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/bigquery.connectionUser'

In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT}  --role='roles/cloudtranslate.user'

In [ ]:
!gcloud projects add-iam-policy-binding bigquerytestdefault --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT}  --role='roles/storage.objectViewer'

In [ ]:
#create object table
try:
  client = bigquery.Client(project=PROJECT_ID)
  sql = """
      CREATE OR REPLACE EXTERNAL TABLE
      `{}.{}.{}`
      WITH CONNECTION `{}.{}` OPTIONS( object_metadata = 'SIMPLE',
      uris = ['gs://data-analytics-demos/TextOCR_images/*']);
      """.format(PROJECT_ID,DATASET_ID, OJBECT_TABLE_NAME, REGION, CONN_NAME )
  query_job = client.query(sql)
  results = query_job.result()

except Exception as e:
  raise Exception(str(e))



In [ ]:
#query object table
try:
  client = bigquery.Client(project=PROJECT_ID)
  sql = """
      SELECT * FROM
      `{PROJECT_ID}.{DATASET_ID}.{OJBECT_TABLE_NAME}` limit 50 ;
      """.format(PROJECT_ID = PROJECT_ID,DATASET_ID = DATASET_ID, OJBECT_TABLE_NAME=OJBECT_TABLE_NAME)
  df = client.query(sql).to_dataframe()

except Exception as e:
  raise Exception(str(e))

df.head(23)

,uri,generation,content_type,size,md5_hash,updated,metadata
0,gs://data-analytics-demos/TextOCR_images/00875...,1680039280565689,image/jpeg,766494,b20ee2f0609a75e4ba95666893a1ba8c,2023-03-28 21:34:40.607000+00:00,[]
1,gs://data-analytics-demos/TextOCR_images/01023...,1680039280553219,image/jpeg,824430,51911a441d0213f3f644b4c092ceb137,2023-03-28 21:34:40.605000+00:00,[]
2,gs://data-analytics-demos/TextOCR_images/0328f...,1680039280563977,image/jpeg,747167,dec21a8a7cb426272eb7cf2739cfde06,2023-03-28 21:34:40.605000+00:00,[]
3,gs://data-analytics-demos/TextOCR_images/04b03...,1680039280568388,image/jpeg,702377,87f32af89141a6269501dd285ac6699c,2023-03-28 21:34:40.608000+00:00,[]
4,gs://data-analytics-demos/TextOCR_images/07901...,1680039280574171,image/jpeg,851695,16886737338012a0e556e8331eba7f8a,2023-03-28 21:34:40.609000+00:00,[]
5,gs://data-analytics-demos/TextOCR_images/08d54...,1680039280676556,image/jpeg,815716,427c6695e732898c8606c1ec2f21cf8f,2023-03-28 21:34:40.708000+00:00,[]
6,gs://data-analytics-demos/TextOCR_images/0a2e7...,1680039280768719,image/jpeg,2894848,6e7fc1b12dc8519d81986c1a60305874,2023-03-28 21:34:40.802000+00:00,[]
7,gs://data-analytics-demos/TextOCR_images/0ba12...,1680039280784257,image/jpeg,768752,a841c7dbbed84076e15d9adbb494d2c8,2023-03-28 21:34:40.815000+00:00,[]
8,gs://data-analytics-demos/TextOCR_images/0d9a4...,1680039280788262,image/jpeg,1170628,1fa4ecc7be62842dec729f61e6508508,2023-03-28 21:34:40.844000+00:00,[]
9,gs://data-analytics-demos/TextOCR_images/0e5fe...,1680039280781157,image/jpeg,725448,d55f2072149d1ccf101841ec691660e1,2023-03-28 21:34:40.815000+00:00,[]


In [ ]:
#create vision api model
try:
  client = bigquery.Client(project=PROJECT_ID)
  sql = """
      CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}.{VISION_MODEL_NAME}`
      REMOTE WITH CONNECTION `{PROJECT_ID}.{REGION}.{CONN_NAME}`
      OPTIONS (
        remote_service_type = 'cloud_ai_vision_v1');
      """.format(PROJECT_ID = PROJECT_ID,DATASET_ID = DATASET_ID, OJBECT_TABLE_NAME=OJBECT_TABLE_NAME, VISION_MODEL_NAME=VISION_MODEL_NAME, REGION=REGION, CONN_NAME=CONN_NAME)
  query_job = client.query(sql)
  results = query_job.result()

except Exception as e:
  raise Exception(str(e))



      CREATE OR REPLACE MODEL `bigquerytestdefault.steveswalker.bqml-cloud-ai-vision`
      REMOTE WITH CONNECTION `bigquerytestdefault.US.conn_cloud_ai`
      OPTIONS (
        remote_service_type = 'cloud_ai_vision_v1');
      


In [ ]:
client = bigquery.Client()

model_id = '{}.{}.{}'.format(PROJECT_ID, DATASET_ID, VISION_MODEL_NAME)

model = client.get_model(model_id)  # Make an API request.

full_model_id = "{}.{}.{}".format(model.project, model.dataset_id, model.model_id)
friendly_name = model.friendly_name
print(
    "Vision Model now available '{}'".format(full_model_id, friendly_name)
)

Vision Model now available 'bigquerytestdefault.steveswalker.bqml-cloud-ai-vision'


In [ ]:
#create translate api model
try:
  client = bigquery.Client(project=PROJECT_ID)
  sql = """
      CREATE OR REPLACE MODEL
      `{}.{}.{}` REMOTE
      WITH CONNECTION `{}.{}.{}`
      OPTIONS ( remote_service_type = 'CLOUD_AI_TRANSLATE_V3');
      """.format(PROJECT_ID,DATASET_ID, TRANSLATE_MODEL_NAME,PROJECT_ID, REGION, CONN_NAME)
  query_job = client.query(sql)
  results = query_job.result()

  model_id = '{}.{}.{}'.format(PROJECT_ID, DATASET_ID, TRANSLATE_MODEL_NAME)

  model = client.get_model(model_id)  # Make an API request.

  full_model_id = "{}.{}.{}".format(model.project, model.dataset_id, model.model_id)
  friendly_name = model.friendly_name
  print(
    "Translate Model now available '{}'".format(full_model_id, friendly_name)
  )

except Exception as e:
  raise Exception(str(e))

In [ ]:
client = bigquery.Client()

model_id = '{}.{}.{}'.format(PROJECT_ID, DATASET_ID, TRANSLATE_MODEL_NAME)

model = client.get_model(model_id)  # Make an API request.

full_model_id = "{}.{}.{}".format(model.project, model.dataset_id, model.model_id)
friendly_name = model.friendly_name
print(
    "Translate Model now available '{}'".format(full_model_id, friendly_name)
)




In [ ]:
try:
  client = bigquery.Client(project=PROJECT_ID)
  sql = """
      CREATE OR REPLACE MODEL
      `{}.{}.{}` REMOTE
      WITH CONNECTION `{}.{}.{}`
      OPTIONS ( remote_service_type = 'cloud_ai_natural_language_v1');
      """.format(PROJECT_ID,DATASET_ID, TEXT_MODEL_NAME,PROJECT_ID, REGION, CONN_NAME)
  query_job = client.query(sql)
  results = query_job.result()

  model_id = '{}.{}.{}'.format(PROJECT_ID, DATASET_ID, TRANSLATE_MODEL_NAME)

  model = client.get_model(model_id)  # Make an API request.

  full_model_id = "{}.{}.{}".format(model.project, model.dataset_id, model.model_id)
  friendly_name = model.friendly_name
  print(
    "Translate Model now available '{}'".format(full_model_id, friendly_name)
  )

except Exception as e:
  raise Exception(str(e))

Translate Model now available 'bigquerytestdefault.steveswalker.bqml-cloud-ai-translate'


In [ ]:
#Use vision ai to get text from images
#then use translate ai to check language of text
try:
  client = bigquery.Client(project=PROJECT_ID)
  sql = """
        CREATE OR REPLACE TABLE
          `{}.{}.temp_image_results` AS
        SELECT
          --this column must be aliased/named 'text_content'
          STRING(ml_annotate_image_result.full_text_annotation.text) AS text_content
        FROM
          ML.ANNOTATE_IMAGE( MODEL `{}.{}.{}`,
            TABLE `{}.{}.{}`,
            STRUCT(['TEXT_DETECTION'] AS vision_features));

        CREATE OR REPLACE TABLE {}.{}.temp_detect_language_results as
        SELECT
          ml_translate_result.languages[0].language_code,
          ml_translate_result.languages[0].confidence, *
        FROM
          ML.TRANSLATE( MODEL `{}.{}.{}`, TABLE `{}.{}.temp_image_results`, STRUCT("detect_language" AS translate_mode));

        SELECT * FROM {}.{}.temp_detect_language_results;
        """.format(PROJECT_ID, DATASET_ID, PROJECT_ID, DATASET_ID, VISION_MODEL_NAME, PROJECT_ID, DATASET_ID, OJBECT_TABLE_NAME, PROJECT_ID, DATASET_ID, PROJECT_ID, DATASET_ID, TRANSLATE_MODEL_NAME, PROJECT_ID,DATASET_ID, PROJECT_ID, DATASET_ID )

  df = client.query(sql).to_dataframe()

except Exception as e:
  raise Exception(str(e))



df.head(20)

,language_code,confidence,ml_translate_result,ml_translate_status,text_content
0,None,None,None,None,None
1,None,None,None,None,None
2,"""en""",0.82351506,"{""languages"":[{""confidence"":0.82351506,""langua...",,THE\nSANOMAN\nOC\nBASED ON THE GRAPHIC NOVELS ...
3,"""en""",0.87622786,"{""languages"":[{""confidence"":0.87622786,""langua...",,A ZEES ORIGINAL\nZEES\nPREMIERES 31ST JULY\nDO...
4,"""en""",0.8939774,"{""languages"":[{""confidence"":0.8939774,""languag...",,MARK MITCHELL\nCuma\nNATE MCFADDEN!\nHOLLYWOOD...
5,"""en""",0.92675847,"{""languages"":[{""confidence"":0.92675847,""langua...",,www.\nCOMING SOON FALL 2018\nMELS... FINE ART ...
6,"""en""",1,"{""languages"":[{""confidence"":1,""language_code"":...",,++\nX\nFAITH BASED\nSALVATION IS RICH\nLONE
7,"""en""",0.46113598,"{""languages"":[{""confidence"":0.46113598,""langua...",,"DONZILLA, INC. PRESENTS:\nRED\nDorulla, Inc. P..."
8,"""en""",0.95719844,"{""languages"":[{""confidence"":0.95719844,""langua...",,doodle
9,"""en""",0.74308014,"{""languages"":[{""confidence"":0.74308014,""langua...",,From The Director of Bhoothnath\n得\nfast\ncali...


In [ ]:
#Use vision ai to get text from images
#then use translate ai to translate text
try:
  client = bigquery.Client(project=PROJECT_ID)
  sql = """
        SELECT text_content, language_code,
          ml_translate_result.translations[0].translated_text
        FROM
          ML.TRANSLATE( MODEL `{}.{}.{}`, TABLE `{}.{}.temp_detect_language_results`, STRUCT("translate_text" as translate_mode, "en" as target_language_code));
        """.format( PROJECT_ID,DATASET_ID, TRANSLATE_MODEL_NAME, PROJECT_ID,DATASET_ID)

  df = client.query(sql).to_dataframe()

except Exception as e:
  raise Exception(str(e))

df.head(50)






        SELECT text_content, language_code,  
          ml_translate_result.translations[0].translated_text
        FROM
          ML.TRANSLATE( MODEL `bigquerytestdefault.steveswalker.bqml-cloud-ai-translate`, TABLE `bigquerytestdefault.steveswalker.temp_detect_language_results`, STRUCT("translate_text" as translate_mode, "en" as target_language_code));
        


,language_code,confidence,ml_translate_result,ml_translate_status,text_content
0,None,None,None,None,None
1,None,None,None,None,None
2,"""en""",0.82351506,"{""languages"":[{""confidence"":0.82351506,""langua...",,THE\nSANOMAN\nOC\nBASED ON THE GRAPHIC NOVELS ...
3,"""en""",0.87622786,"{""languages"":[{""confidence"":0.87622786,""langua...",,A ZEES ORIGINAL\nZEES\nPREMIERES 31ST JULY\nDO...
4,"""en""",0.8939774,"{""languages"":[{""confidence"":0.8939774,""languag...",,MARK MITCHELL\nCuma\nNATE MCFADDEN!\nHOLLYWOOD...
5,"""en""",0.92675847,"{""languages"":[{""confidence"":0.92675847,""langua...",,www.\nCOMING SOON FALL 2018\nMELS... FINE ART ...
6,"""en""",1,"{""languages"":[{""confidence"":1,""language_code"":...",,++\nX\nFAITH BASED\nSALVATION IS RICH\nLONE
7,"""en""",0.46113598,"{""languages"":[{""confidence"":0.46113598,""langua...",,"DONZILLA, INC. PRESENTS:\nRED\nDorulla, Inc. P..."
8,"""en""",0.95719844,"{""languages"":[{""confidence"":0.95719844,""langua...",,doodle
9,"""en""",0.74308014,"{""languages"":[{""confidence"":0.74308014,""langua...",,From The Director of Bhoothnath\n得\nfast\ncali...


In [ ]:
#create object table
try:
  client = bigquery.Client(project=PROJECT_ID)
  sql = """
      SELECT
      text_content AS `Original Input`,
      ml_understand_text_result.categories[0].name AS `Classified Name`,
      ml_understand_text_result.categories[0].confidence AS `Confidence`
      FROM
      ML.UNDERSTAND_TEXT( MODEL `{}.{}.{}`,
        TABLE `{}.{}.temp_detect_language_results`,
        STRUCT('classify_text' AS nlu_option))
        where  ml_understand_text_result.categories[0].name is not null ;
      """.format(PROJECT_ID,DATASET_ID, TEXT_MODEL_NAME, PROJECT_ID,DATASET_ID )
  df = client.query(sql).to_dataframe()

except Exception as e:
  raise Exception(str(e))



In [ ]:
df.head()

,Original Input,Classified Name,Confidence
0,MARK MITCHELL\nCuma\nNATE MCFADDEN!\nHOLLYWOOD...,"""/Arts & Entertainment""",0.76999998
1,"DONZILLA, INC. PRESENTS:\nRED\nDorulla, Inc. P...","""/Arts & Entertainment""",0.73000002
2,MICHAEL LAYCI\nJAKE\nO'DAY BOYACK BARTON\nK\nD...,"""/Arts & Entertainment""",0.88999999
3,VERONICA DIAZ\nJESSE C\nCARRANZA BOYD\nthe\nFI...,"""/Arts & Entertainment""",0.67000002
4,GOLDEN WAY MEDIA FILMS\nPRESENTS\nCARINA\nTOR ...,"""/Arts & Entertainment""",0.75999999
